# Group Assignment

1. We can define the second difference matrix as follows ($N=5$ for example):
   $$
    A = \begin{pmatrix}
    2 &-1  &  & &\\
     -1 & 2 &-1  & \\
     & -1 & 2 &-1  \\
     & & -1  &2 &-1 \\
     & & & -1 & 2
    \end{pmatrix}
    $$
    a) Let the true solution $x_a$ be generated by the normal distributed random numbers, thus $b = Ax$  
    b) The convergence condition can be set as $||x_{n+1} − x_n|| \leq \epsilon$ for some small value of $\epsilon$. Choose your own $\epsilon$, try to modify the sample script `jacobi_method.ipynb` to count how many loop iterations occur before convergence. How does it change with the change of $N$?  
    c) The weighted Jacobi method uses a parameter $\omega$ to compute the iteration as
   $$
    \mathbf {x} ^{(k+1)}=\left(1-\omega \right)\mathbf {x} ^{(k)}+ \omega\mathbf {x} ^{(k+1)}_{\text{Jacobi}}=\left(1-\omega \right)\mathbf {x} ^{(k)}+ \omega D^{-1}(\mathbf {b} -(L+U)\mathbf {x} ^{(k)})
   $$
    with $\omega =2/3$ being the usual choice. Modify your script to implement the weighted Jacobi method. Does it converge faster than the Jacobi method?
    Hint: You can read *Jacobi_method_Wikipedia.pdf* for your reference. 

In [1]:
import numpy as np
import random

def Generate_A_b(N):     # 定义一个产生我们要求的A,x_a,b的函数
    I = np.eye(N)
    L = np.zeros((N,N))
    for i in range(1,N):
        L[i,i-1] = 1
    U = L.T
    A = 2*I - L - U
    x_a = np.zeros((N,1))
    for i in range(N):
        x_a[i,0] = random.normalvariate(0,1)
    b = A@x_a
    return A,x_a,b

(A,x_a,b) = Generate_A_b(5)
print(f'A = \n{A}')
print(f'x_a =\n {x_a}')
print(f'b = \n{b}')


A = 
[[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]
x_a =
 [[ 0.20019286]
 [ 0.91865424]
 [ 2.31280488]
 [-0.39190734]
 [-0.30371174]]
b = 
[[-0.51826851]
 [-0.67568927]
 [ 4.09886287]
 [-2.79290783]
 [-0.21551613]]


In [8]:
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot

def jacobi(A, b, eps = 1e-5, N = 1000 ,x=None):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create an initial guess if needed                                                     
    if x is None:
        x = np.ones((np.shape(A)[0],1))
    # Create a vector of the diagonal elements of A
    # and subtract them from A

    D = np.matrix(diag(A)).T
    R = A - diagflat(D)
    
    xlast = x
    for i in range(N):
        x = (b - dot(R,x)) / D
        if np.linalg.norm(x-xlast) < eps:
            break
        xlast = x
    
    return x, i+1 

NN = [5,7,10,15]        # test the iteration times when N = 5, 7, 10, 15
for N in NN:
    print(f'for N = {N}')
    (A,x_a,b) = Generate_A_b(N)
    if N == 5:
        store = (A,x_a,b)
    print(f'A = \n{A}')
    print(f'x_a =\n {x_a}')
    print(f'b = \n{b}')
    sol = jacobi(A,b)
    print("x:"); pprint(sol[0])
    print(f'iteration times: {sol[1]}')
    print(60*'-')

for N = 5
A = 
[[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]
x_a =
 [[ 1.83366801]
 [ 0.40732021]
 [ 1.24706754]
 [-1.06900468]
 [ 0.1894497 ]]
b = 
[[ 3.26001581]
 [-2.26609513]
 [ 3.15581955]
 [-3.57452659]
 [ 1.44790407]]
x:
matrix([[ 1.83367013],
        [ 0.4073198 ],
        [ 1.24707177],
        [-1.06900509],
        [ 0.18945181]])
iteration times: 89
------------------------------------------------------------
for N = 7
A = 
[[ 2. -1.  0.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.]
 [ 0.  0. -1.  2. -1.  0.  0.]
 [ 0.  0.  0. -1.  2. -1.  0.]
 [ 0.  0.  0.  0. -1.  2. -1.]
 [ 0.  0.  0.  0.  0. -1.  2.]]
x_a =
 [[-0.38260216]
 [ 1.32405375]
 [ 0.89726224]
 [ 0.90194769]
 [ 0.25562036]
 [-2.01371202]
 [-0.50865811]]
b = 
[[-2.08925807]
 [ 2.13344742]
 [-0.43147696]
 [ 0.65101279]
 [ 1.62300505]
 [-3.7743863 ]
 [ 0.9963958 ]]
x:
matrix([[-0.3825829 ],
        [ 1.32408737],

Obviously, iteration times grows with $N$

In [13]:
def jacobi_weight(A, b, weight, eps = 1e-5, N = 1000 ,x=None):
    # Create an initial guess if needed                                                     
    if x is None:
        x = np.ones((np.shape(A)[0],1))
    # Create a vector of the diagonal elements of A
    # and subtract them from A

    D = np.matrix(diag(A)).T
    R = A - diagflat(D)
    
    xlast = x
    for i in range(N):
        x = (1 - weight) * x + weight * (b - dot(R,x)) / D
        if np.linalg.norm(x-xlast) < eps:
            break
        xlast = x

    return x, i+1 

(A,b) = store[0],store[2]
sol = jacobi_weight(A,b,2.0/3)

print("x:"); pprint(sol[0])
print(f'iteration times: {sol[1]}')

x:
matrix([[ 1.83369486],
        [ 0.40736672],
        [ 1.24712124],
        [-1.06895817],
        [ 0.18947655]])
iteration times: 101


To get the same precision $|x_{n+1} - x_{n}|\lt 1e-5$, jacobi method iteration times is **89**, the weighted Jacobi method interation times is **101**, so the weighted Jacobi method isn't faster

2. Apply the Gauss-Seidel method to solve the linear equations given in Problem 1.
   a)  Choose your own $\epsilon$, try to modify the sample script `gauss_seidel_method.ipynb` to count the executing time before convergence. How does it change with the change of $N$? 
   b) Does Gauss-Seidel method converge, as expected, faster than the Jacobi method?
   Hint: You can read *Gauss_Seidel_method_Wikipedia.pdf* for your reference. 

3. The method of successive over-relaxation (SOR) is a variant of the Gauss–Seidel method for solving a linear system of equations, resulting in faster convergence, where the iterative is defined as
   $$
   \begin{aligned}
   \mathbf{x}^{(k+1)}&= (1-\omega)\mathbf{x}^{(k)}+\omega \mathbf{x}^{(k+1)}_{\text{Gauss-Seidel}}\\
   x_i^{(k+1)}&=(1-\omega)x_i^{(k)}+\frac{\omega}{a_{ii}}\left(b_i-\displaystyle\sum_{j<i}a_{ij}x_j^{(k+1)}-\displaystyle\sum_{j>i}a_{ij}x_j^{(k)}\right)
   \end{aligned}
   $$
   where $0<\omega <2$. $\omega >1$ is called the relaxation factor. 
   Consider the linear system as follows:
   $$
   \begin{cases}
   3x_1 - x_2+ x_3 = -1\\
   -x_1 +3x_2 -x_3 = 7\\
   x_1 - x_2 +3x_3 = -7\\
      \end{cases}
   $$
   a) Set your own $\epsilon$, try to modify the sample script `gauss_seidel_method.ipynb` to implement the SOR method.
   b) Can you find the optimal value of $\omega$ empirically? Please compare your result with the theoretical prediction - the optimal relaxation parameter $\omega$ is given by
    $$
    \omega_\text{opt} = 1+\left(\frac{\lambda_J}{1+\sqrt{1-\lambda_J^2}}\right)^2 
    $$
    $\lambda_J$ is the spectral radius calculated  in the Jacobi method.   
    Hint: You can read *SOR_method_Wikipedia.pdf* for your reference. 
   

4. One application of the conjugate gradient method is to solve the normal equation to find the least-square solution of an (over-determined) equation system $\mathbf{A}\mathbf{x}=\mathbf{b}$, where the coefficient matrix $\mathbf{A}$ is $M$ by $N$ of rank $n$, i.e., $M\geq N=n$.The normal equation of this system is 
$$
\mathbf{A}^{T}\mathbf{A}\mathbf{x}=\mathbf{A}^{T}\mathbf{b}% 
$$
Here $\mathbf{A}^{T}\mathbf{A}$ is an $N$ by $N$ symmetric, positive definite matrix. When $A$ is a large-sized sparse symmetrical matrix, conjugate gradient method is also preferred. 
Modify the sample script  `conj_grad.ipynb` to solve Problem 1. Please compare your results with the results from the Jacobi method and the Cholesky method. 

5. Read Example 7.2[^1] and the sample script `power_method_eig.ipynb`. The problem is to find the largest eigenvalue of matrix $A$:
   $$
   A = \begin{pmatrix}
   4 & -i & 2 \\
   i & 2 & 2+7i \\
   2 & 2-7i & -2
   \end{pmatrix}
   $$
  a) Try modifying the above example to count how many loop iterations occur before convergence. Is it larger or smaller than you had expected?  
  b) An alternative to using the Rayleigh quotient to determine whether convergence is achieved is the convergence criterion 
   $$
   ||x_{n+1} − x_n|| \leq \epsilon
   $$
    This is especially useful if we do not need to compute the eigenvalue, and can save some computational time. Modify the code above to use this convergence criterion, and compare the overall wall time to the original version.

6. We can use the normalised power method above on $A^{−1}$ to calculate $\lambda_N$ — the smallest magnitude eigenvalue of $A$, and the dominant eigenvalue of $A^{−1}$. In practice, however, matrix inversion is a difficult numerical computation, is prone to error, and can be quite unstable. A more common approach than direct matrix inversion is to view the inverse power iteration as a system of linear equations to be solved involving $A$:  
Step 1. Choose a normalised starting vector $x_0$ where $||x_0|| = 1$;   
Step 2. Solve the system of linear equations $Ax_{n+1} = x_n$ to determine the next vector in the iteration, and normalise this vector;  
Step 3. Repeat step 2 until convergence is achieved, $||x_{n+1} − x_n|| \leq \epsilon$ for some small value of $\epsilon$. The Rayleigh quotient $x_n^\dagger Ax_n$ will provide the value of the smallest magnitude eigenvector of $A$.    
Modify the sample script to find the smallest eigenvalue of matrix $A$.

[^1]: Joshua Izaac, Jingbo Wang, Computational Quantum Mechanics, Springer, 2018

In [3]:
import numpy as np

def conjugate_residual(A, b, x=None):  
    """
    conjugate residual method to solve Hermitian matrix
    \(this method come from Wikipedia, url: https://en.wikipedia.org/wiki/Conjugate_residual_method\)
    """
    # if use Gauss method, it's equal to calculate the inverse of A
    n = len(b)
    if not x:
        x = np.ones(n)

    x = x
    r = b - np.dot(A,x)
    p = r
    
    for i in range( 2 * n ):
        rAr = np.dot(np.conj(r),np.dot(A,r))
        Ap = np.dot(A,p)
        alpha = rAr / np.dot( np.conj(Ap) , Ap )
        
        x = x + alpha * p
        r = r - alpha * Ap

        rAr_plus_one = np.dot( np.conj(r) ,np.dot(A,r))
        beta = rAr_plus_one / rAr

        
        if abs(np.linalg.norm(r)) < 1e-5:
            break

        p = r + beta * p

    return x

# function to calculate the Rayleigh quotient
def rayleigh_quotient(A,x):
    return np.dot(x, np.dot(A, x))/np.dot(x,x)

# function to normalise a vector
def normalise(x,eps=1e-10):
    N = np.sqrt(np.sum(abs(x)**2))
    if N < eps: # in case it is the zero vector!
        return x
    else:
        return x/N

A = np.array([[4, -1j, 2],
        [1j, 2, 2+7j],
        [2, 2-7j, -2]])

# choose the starting vector
x = normalise(np.array([1, 1, 1]))
RQnew = rayleigh_quotient(A,x)
RQold = 0

# perform the power iteration
while np.abs(RQnew-RQold) > 1e-6:
    RQold = RQnew
    x = normalise(conjugate_residual(A, x))
    RQnew = rayleigh_quotient(A, x)

print("Smallest eigenvalue: {:.5f}".format(RQnew))
print("Eigenvector correspond:\n",x)

Smallest eigenvalue: 3.18960-0.00000j
Eigenvector correspond:
 [ 0.76568347+0.48507841j  0.1333804 -0.35521551j -0.13264612-0.1298629j ]
